In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Input, LSTM, GRU, Bidirectional, SimpleRNN, Embedding, TimeDistributed
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences, to_categorical
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, brown, treebank
#import re
#from nltk.stem import SnowballStemmer, WordNetLemmatizer
#from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [2]:
import nltk
nltk.download('treebank')

[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\madas\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!


True

In [3]:
brown.sents()

[['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', "Atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.'], ['The', 'jury', 'further', 'said', 'in', 'term-end', 'presentments', 'that', 'the', 'City', 'Executive', 'Committee', ',', 'which', 'had', 'over-all', 'charge', 'of', 'the', 'election', ',', '``', 'deserves', 'the', 'praise', 'and', 'thanks', 'of', 'the', 'City', 'of', 'Atlanta', "''", 'for', 'the', 'manner', 'in', 'which', 'the', 'election', 'was', 'conducted', '.'], ...]

In [4]:
brown

<CategorizedTaggedCorpusReader in 'C:\\Users\\madas\\AppData\\Roaming\\nltk_data\\corpora\\brown'>

In [5]:
len(brown.sents())

57340

In [6]:
" ".join(brown.sents()[0])

"The Fulton County Grand Jury said Friday an investigation of Atlanta's recent primary election produced `` no evidence '' that any irregularities took place ."

In [7]:
" ".join(brown.sents()[1])

"The jury further said in term-end presentments that the City Executive Committee , which had over-all charge of the election , `` deserves the praise and thanks of the City of Atlanta '' for the manner in which the election was conducted ."

In [8]:
treebank.sents()

[['Pierre', 'Vinken', ',', '61', 'years', 'old', ',', 'will', 'join', 'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'Nov.', '29', '.'], ['Mr.', 'Vinken', 'is', 'chairman', 'of', 'Elsevier', 'N.V.', ',', 'the', 'Dutch', 'publishing', 'group', '.'], ...]

In [9]:
len(treebank.sents())

3914

In [10]:
" ".join(treebank.sents()[1])

'Mr. Vinken is chairman of Elsevier N.V. , the Dutch publishing group .'

In [11]:
" ".join(brown.sents()[0])

"The Fulton County Grand Jury said Friday an investigation of Atlanta's recent primary election produced `` no evidence '' that any irregularities took place ."

In [12]:
brown.tagged_sents(tagset="universal")[0] #universal Database contain huge tags

[('The', 'DET'),
 ('Fulton', 'NOUN'),
 ('County', 'NOUN'),
 ('Grand', 'ADJ'),
 ('Jury', 'NOUN'),
 ('said', 'VERB'),
 ('Friday', 'NOUN'),
 ('an', 'DET'),
 ('investigation', 'NOUN'),
 ('of', 'ADP'),
 ("Atlanta's", 'NOUN'),
 ('recent', 'ADJ'),
 ('primary', 'NOUN'),
 ('election', 'NOUN'),
 ('produced', 'VERB'),
 ('``', '.'),
 ('no', 'DET'),
 ('evidence', 'NOUN'),
 ("''", '.'),
 ('that', 'ADP'),
 ('any', 'DET'),
 ('irregularities', 'NOUN'),
 ('took', 'VERB'),
 ('place', 'NOUN'),
 ('.', '.')]

In [13]:
import nltk
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\madas\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [14]:
" ".join(treebank.sents()[0])

'Pierre Vinken , 61 years old , will join the board as a nonexecutive director Nov. 29 .'

In [15]:
treebank.tagged_sents(tagset="universal")[0]

[('Pierre', 'NOUN'),
 ('Vinken', 'NOUN'),
 (',', '.'),
 ('61', 'NUM'),
 ('years', 'NOUN'),
 ('old', 'ADJ'),
 (',', '.'),
 ('will', 'VERB'),
 ('join', 'VERB'),
 ('the', 'DET'),
 ('board', 'NOUN'),
 ('as', 'ADP'),
 ('a', 'DET'),
 ('nonexecutive', 'ADJ'),
 ('director', 'NOUN'),
 ('Nov.', 'NOUN'),
 ('29', 'NUM'),
 ('.', '.')]

In [16]:
d1 = treebank.tagged_sents(tagset="universal")

In [17]:
d2 = brown.tagged_sents(tagset="universal")

In [18]:
df = d1+d2

In [19]:
len(df)

61254

In [20]:
pd.DataFrame(df)

,0,1,2,3,4,5,6,7,8,9,...,261,262,263,264,265,266,267,268,269,270
0,"(Pierre, NOUN)","(Vinken, NOUN)","(,, .)","(61, NUM)","(years, NOUN)","(old, ADJ)","(,, .)","(will, VERB)","(join, VERB)","(the, DET)",...,None,None,None,None,None,None,None,None,None,None
1,"(Mr., NOUN)","(Vinken, NOUN)","(is, VERB)","(chairman, NOUN)","(of, ADP)","(Elsevier, NOUN)","(N.V., NOUN)","(,, .)","(the, DET)","(Dutch, NOUN)",...,None,None,None,None,None,None,None,None,None,None
2,"(Rudolph, NOUN)","(Agnew, NOUN)","(,, .)","(55, NUM)","(years, NOUN)","(old, ADJ)","(and, CONJ)","(former, ADJ)","(chairman, NOUN)","(of, ADP)",...,None,None,None,None,None,None,None,None,None,None
3,"(A, DET)","(form, NOUN)","(of, ADP)","(asbestos, NOUN)","(once, ADV)","(used, VERB)","(*, X)","(*, X)","(to, PRT)","(make, VERB)",...,None,None,None,None,None,None,None,None,None,None
4,"(The, DET)","(asbestos, NOUN)","(fiber, NOUN)","(,, .)","(crocidolite, NOUN)","(,, .)","(is, VERB)","(unusually, ADV)","(resilient, ADJ)","(once, ADP)",...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61249,"(S., NOUN)","(J., NOUN)","(Perelman, NOUN)",None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
61250,"(revulsion, NOUN)","(in, ADP)","(the, DET)","(desert, NOUN)",None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
61251,"(the, DET)","(doors, NOUN)","(of, ADP)","(the, DET)","(D, NOUN)","(train, NOUN)","(slid, VERB)","(shut, VERB)","(,, .)","(and, CONJ)",...,None,None,None,None,None,None,None,None,None,None
61252,"(She, PRON)","(was, VERB)","(a, DET)","(living, VERB)","(doll, NOUN)","(and, CONJ)","(no, DET)","(mistake, NOUN)","(--, .)","(the, DET)",...,None,None,None,None,None,None,None,None,None,None


In [21]:
d1

[[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')], [('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')], ...]

In [22]:
x = []
y = []
for i in df:
    x1 = []
    x2 = []
    for j in i:
        x1.append(j[0])
        x2.append(j[1])
    x.append(x1)
    y.append(x2)   

In [23]:
len(x)

61254

In [24]:
len(y)

61254

In [25]:
x

[['Pierre',
  'Vinken',
  ',',
  '61',
  'years',
  'old',
  ',',
  'will',
  'join',
  'the',
  'board',
  'as',
  'a',
  'nonexecutive',
  'director',
  'Nov.',
  '29',
  '.'],
 ['Mr.',
  'Vinken',
  'is',
  'chairman',
  'of',
  'Elsevier',
  'N.V.',
  ',',
  'the',
  'Dutch',
  'publishing',
  'group',
  '.'],
 ['Rudolph',
  'Agnew',
  ',',
  '55',
  'years',
  'old',
  'and',
  'former',
  'chairman',
  'of',
  'Consolidated',
  'Gold',
  'Fields',
  'PLC',
  ',',
  'was',
  'named',
  '*-1',
  'a',
  'nonexecutive',
  'director',
  'of',
  'this',
  'British',
  'industrial',
  'conglomerate',
  '.'],
 ['A',
  'form',
  'of',
  'asbestos',
  'once',
  'used',
  '*',
  '*',
  'to',
  'make',
  'Kent',
  'cigarette',
  'filters',
  'has',
  'caused',
  'a',
  'high',
  'percentage',
  'of',
  'cancer',
  'deaths',
  'among',
  'a',
  'group',
  'of',
  'workers',
  'exposed',
  '*',
  'to',
  'it',
  'more',
  'than',
  '30',
  'years',
  'ago',
  ',',
  'researchers',
  'reported'

In [26]:
y

[['NOUN',
  'NOUN',
  '.',
  'NUM',
  'NOUN',
  'ADJ',
  '.',
  'VERB',
  'VERB',
  'DET',
  'NOUN',
  'ADP',
  'DET',
  'ADJ',
  'NOUN',
  'NOUN',
  'NUM',
  '.'],
 ['NOUN',
  'NOUN',
  'VERB',
  'NOUN',
  'ADP',
  'NOUN',
  'NOUN',
  '.',
  'DET',
  'NOUN',
  'VERB',
  'NOUN',
  '.'],
 ['NOUN',
  'NOUN',
  '.',
  'NUM',
  'NOUN',
  'ADJ',
  'CONJ',
  'ADJ',
  'NOUN',
  'ADP',
  'NOUN',
  'NOUN',
  'NOUN',
  'NOUN',
  '.',
  'VERB',
  'VERB',
  'X',
  'DET',
  'ADJ',
  'NOUN',
  'ADP',
  'DET',
  'ADJ',
  'ADJ',
  'NOUN',
  '.'],
 ['DET',
  'NOUN',
  'ADP',
  'NOUN',
  'ADV',
  'VERB',
  'X',
  'X',
  'PRT',
  'VERB',
  'NOUN',
  'NOUN',
  'NOUN',
  'VERB',
  'VERB',
  'DET',
  'ADJ',
  'NOUN',
  'ADP',
  'NOUN',
  'NOUN',
  'ADP',
  'DET',
  'NOUN',
  'ADP',
  'NOUN',
  'VERB',
  'X',
  'PRT',
  'PRON',
  'ADV',
  'ADP',
  'NUM',
  'NOUN',
  'ADP',
  '.',
  'NOUN',
  'VERB',
  'X',
  'X',
  '.'],
 ['DET',
  'NOUN',
  'NOUN',
  '.',
  'NOUN',
  '.',
  'VERB',
  'ADV',
  'ADJ',
  'ADP'

In [27]:
c = 0
for i in range(len(x)):
    if len(x[i]) == len(y[i]):
        pass
    else:
        c = +1
c

0

In [28]:
#vocabluary size 
voc = []
for i in x:
    for word in i:
        if word.lower() not in voc:
            voc.append(word.lower())

In [29]:
len(voc)

53232

In [30]:
voc

['pierre',
 'vinken',
 ',',
 '61',
 'years',
 'old',
 'will',
 'join',
 'the',
 'board',
 'as',
 'a',
 'nonexecutive',
 'director',
 'nov.',
 '29',
 '.',
 'mr.',
 'is',
 'chairman',
 'of',
 'elsevier',
 'n.v.',
 'dutch',
 'publishing',
 'group',
 'rudolph',
 'agnew',
 '55',
 'and',
 'former',
 'consolidated',
 'gold',
 'fields',
 'plc',
 'was',
 'named',
 '*-1',
 'this',
 'british',
 'industrial',
 'conglomerate',
 'form',
 'asbestos',
 'once',
 'used',
 '*',
 'to',
 'make',
 'kent',
 'cigarette',
 'filters',
 'has',
 'caused',
 'high',
 'percentage',
 'cancer',
 'deaths',
 'among',
 'workers',
 'exposed',
 'it',
 'more',
 'than',
 '30',
 'ago',
 'researchers',
 'reported',
 '0',
 '*t*-1',
 'fiber',
 'crocidolite',
 'unusually',
 'resilient',
 'enters',
 'lungs',
 'with',
 'even',
 'brief',
 'exposures',
 'causing',
 'symptoms',
 'that',
 'show',
 'up',
 'decades',
 'later',
 'said',
 '*t*-2',
 'lorillard',
 'inc.',
 'unit',
 'new',
 'york-based',
 'loews',
 'corp.',
 'makes',
 'cigare

In [31]:
tags = []
for i in y:
    for tag in i:
        if tag.lower() not in tags:
            tags.append(tag.lower())

In [32]:
len(tags)

12

In [33]:
tags

['noun',
 '.',
 'num',
 'adj',
 'verb',
 'det',
 'adp',
 'conj',
 'x',
 'adv',
 'prt',
 'pron']

In [34]:
tk_x = Tokenizer()
tk_x.fit_on_texts(x)

tk_y = Tokenizer()
tk_y.fit_on_texts(y)

In [35]:
tk_x.document_count

61254

In [36]:
tk_x.word_counts

OrderedDict([('pierre', 18),
             ('vinken', 2),
             (',', 63219),
             ('61', 9),
             ('years', 1065),
             ('old', 693),
             ('will', 2526),
             ('join', 69),
             ('the', 74735),
             ('board', 312),
             ('as', 7668),
             ('a', 25183),
             ('nonexecutive', 5),
             ('director', 133),
             ('nov.', 45),
             ('29', 28),
             ('.', 53174),
             ('mr.', 1219),
             ('is', 10781),
             ('chairman', 124),
             ('of', 38737),
             ('elsevier', 1),
             ('n.v.', 3),
             ('dutch', 18),
             ('publishing', 32),
             ('group', 459),
             ('rudolph', 6),
             ('agnew', 1),
             ('55', 19),
             ('and', 30409),
             ('former', 150),
             ('consolidated', 8),
             ('gold', 63),
             ('fields', 85),
             ('plc', 13),
    

In [37]:
tk_x.index_word

{1: 'the',
 2: ',',
 3: '.',
 4: 'of',
 5: 'and',
 6: 'to',
 7: 'a',
 8: 'in',
 9: 'that',
 10: 'is',
 11: 'for',
 12: 'was',
 13: 'he',
 14: '``',
 15: "''",
 16: 'it',
 17: 'with',
 18: 'as',
 19: 'on',
 20: 'his',
 21: 'be',
 22: 'at',
 23: 'by',
 24: ';',
 25: 'this',
 26: 'had',
 27: 'i',
 28: 'are',
 29: 'from',
 30: 'not',
 31: '?',
 32: 'but',
 33: 'or',
 34: 'have',
 35: 'an',
 36: 'they',
 37: 'which',
 38: '--',
 39: 'were',
 40: 'one',
 41: 'you',
 42: 'all',
 43: 'her',
 44: 'she',
 45: 'would',
 46: 'their',
 47: 'there',
 48: 'has',
 49: 'we',
 50: 'him',
 51: 'been',
 52: 'said',
 53: 'will',
 54: ')',
 55: '(',
 56: 'when',
 57: 'more',
 58: 'who',
 59: 'if',
 60: 'no',
 61: 'its',
 62: 'out',
 63: 'so',
 64: 'up',
 65: 'about',
 66: 'what',
 67: 'than',
 68: 'new',
 69: ':',
 70: 'into',
 71: 'can',
 72: 'them',
 73: 'other',
 74: 'only',
 75: 'some',
 76: 'could',
 77: 'time',
 78: 'these',
 79: '!',
 80: 'two',
 81: 'may',
 82: 'first',
 83: 'any',
 84: 'do',
 85: '

In [38]:
len(tk_x.index_word)

53232

In [39]:
tk_y.document_count

61254

In [40]:
tk_y.word_counts

OrderedDict([('noun', 304425),
             ('.', 159280),
             ('num', 18420),
             ('adj', 90118),
             ('verb', 196314),
             ('det', 145744),
             ('adp', 154623),
             ('conj', 40416),
             ('x', 7999),
             ('adv', 59410),
             ('prt', 33048),
             ('pron', 52071)])

In [41]:
len(tk_y.index_word)

12

In [42]:
x_d = tk_x.texts_to_sequences(x)
y_d = tk_y.texts_to_sequences(y)

In [43]:
final_xd = pad_sequences(x_d, padding="post")
final_yd = pad_sequences(y_d, padding="post")

In [44]:
final_xd.shape

(61254, 271)

In [45]:
final_yd.shape

(61254, 271)

In [46]:
final_xd

array([[ 5594, 21728,     2, ...,     0,     0,     0],
       [   96, 21728,    10, ...,     0,     0,     0],
       [11765, 29497,     2, ...,     0,     0,     0],
       ...,
       [    1,  3243,     4, ...,     0,     0,     0],
       [   44,    12,     7, ...,     0,     0,     0],
       [   29,    66,    27, ...,     0,     0,     0]])

In [47]:
final_yd

array([[1, 1, 3, ..., 0, 0, 0],
       [1, 1, 2, ..., 0, 0, 0],
       [1, 1, 3, ..., 0, 0, 0],
       ...,
       [5, 1, 4, ..., 0, 0, 0],
       [8, 2, 5, ..., 0, 0, 0],
       [4, 5, 8, ..., 0, 0, 0]])

In [48]:
to_categorical(final_yd).shape

(61254, 271, 13)

In [49]:
final_y = to_categorical(final_yd)

In [50]:
final_y

array([[[0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0.

In [52]:
model = Sequential()
model.add(Input(shape= (271,)))
model.add(Embedding(input_dim=53232+1, output_dim=5))
model.add(SimpleRNN(units=50, return_sequences=True))
model.add(Dense(units=13, activation='softmax'))

In [53]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 271, 5)              │         266,165 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ (None, 271, 50)             │           2,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 271, 13)             │             663 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 269,628 (1.03 MB)

 Trainable params: 269,628 (1.03 MB)

 Non-trainable params: 0 (0.00 B)

In [54]:
model = Sequential()
model.add(Input(shape= (271,)))
model.add(Embedding(input_dim=53232+1, output_dim=5))
model.add(SimpleRNN(units=50, return_sequences=True))
model.add(TimeDistributed(Dense(units=13, activation='softmax',)))

In [55]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (None, 271, 5)              │         266,165 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_1 (SimpleRNN)             │ (None, 271, 50)             │           2,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed (TimeDistributed)   │ (None, 271, 13)             │             663 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 269,628 (1.03 MB)

 Trainable params: 269,628 (1.03 MB)

 Non-trainable params: 0 (0.00 B)

In [56]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [57]:
model.fit(final_xd, final_y, batch_size=32, epochs=10, validation_split=0.2)

Epoch 1/10
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 371s 219ms/step - accuracy: 0.9287 - loss: 0.2695 - val_accuracy: 0.9886 - val_loss: 0.0383
Epoch 2/10
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 330s 215ms/step - accuracy: 0.9906 - loss: 0.0344 - val_accuracy: 0.9960 - val_loss: 0.0131
Epoch 3/10
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 330s 215ms/step - accuracy: 0.9960 - loss: 0.0131 - val_accuracy: 0.9964 - val_loss: 0.0106
Epoch 4/10
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 333s 217ms/step - accuracy: 0.9967 - loss: 0.0096 - val_accuracy: 0.9965 - val_loss: 0.0099
Epoch 5/10
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 453s 296ms/step - accuracy: 0.9971 - loss: 0.0082 - val_accuracy: 0.9967 - val_loss: 0.0095
Epoch 6/10
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 328s 214ms/step - accuracy: 0.9974 - loss: 0.0072 - val_accuracy: 0.9968 - val_loss: 0.0095
Epoch 7/10
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 328s 214ms/step - accuracy: 0.9976 - loss: 0.0066 - val_accuracy: 0.9968 - val_loss: 0.0095
Epoch 8/10
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 1294s 845ms/step - a

In [58]:
tex = ["msdhoni loves playing cricket and he is from india"]

In [59]:
tk_x.texts_to_sequences(tex)

[[5514, 1116, 20351, 5, 13, 10, 29, 2080]]

In [60]:
tk_x.sequences_to_texts(tk_x.texts_to_sequences(tex))

['loves playing cricket and he is from india']

In [62]:
test = pad_sequences(tk_x.texts_to_sequences(tex), maxlen=271, padding="post")

In [63]:
model.predict(test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step


array([[[3.10582527e-06, 3.63095687e-03, 9.96154487e-01, ...,
         1.08638422e-04, 2.37264572e-08, 4.33685309e-05],
        [4.50273660e-07, 8.79161153e-03, 9.91010904e-01, ...,
         9.34305644e-05, 1.54458135e-08, 2.09682876e-05],
        [2.38675216e-06, 9.69510138e-01, 6.54762378e-04, ...,
         5.32865779e-05, 2.18339960e-06, 2.73918994e-02],
        ...,
        [9.99999523e-01, 1.04449505e-14, 1.66090022e-08, ...,
         2.15787777e-10, 2.82967221e-07, 8.96902037e-08],
        [9.99999523e-01, 1.04449505e-14, 1.66090022e-08, ...,
         2.15787777e-10, 2.82967221e-07, 8.96902037e-08],
        [9.99999523e-01, 1.04449505e-14, 1.66090022e-08, ...,
         2.15787777e-10, 2.82967221e-07, 8.96902037e-08]]], dtype=float32)

In [64]:
np.argmax(model.predict(test)[0],axis=1).shape

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step


(271,)

In [65]:
np.argmax(model.predict(test)[0],axis=1)[np.argmax(model.predict(test)[0],axis=1)!=0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step


array([2, 2, 1, 9, 8, 2, 4, 1], dtype=int64)

In [66]:
np.argmax(tk_x.texts_to_sequences(tex))

2

In [67]:
tk_y.sequences_to_texts([[2, 2, 1, 9, 8, 2, 4, 1]])

['verb verb noun conj pron verb adp noun']

In [68]:
tex1 = ["The Fulton County Grand Jury said Friday an investigation of Atlanta's recent primary election produced `` no evidence '' that any irregularities took place ."]

In [69]:
test1 = pad_sequences(tk_x.texts_to_sequences(tex1), maxlen=271, padding="post")

In [72]:
np.argmax(model.predict(test1)[0],axis=1)[np.argmax(model.predict(test1)[0],axis=1)!=0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step


array([5, 1, 1, 6, 1, 2, 1, 5, 1, 4, 1, 6, 6, 1, 2, 5, 1, 3, 4, 5, 1, 2,
       1], dtype=int64)

In [73]:
tk_y.sequences_to_texts([[5, 1, 1, 6, 1, 2, 1, 5, 1, 4, 1, 6, 6, 1, 2, 5, 1, 3, 4, 5, 1, 2,1]])

['det noun noun adj noun verb noun det noun adp noun adj adj noun verb det noun . adp det noun verb noun']

In [74]:
brown.tagged_sents(tagset="universal")[0]

[('The', 'DET'),
 ('Fulton', 'NOUN'),
 ('County', 'NOUN'),
 ('Grand', 'ADJ'),
 ('Jury', 'NOUN'),
 ('said', 'VERB'),
 ('Friday', 'NOUN'),
 ('an', 'DET'),
 ('investigation', 'NOUN'),
 ('of', 'ADP'),
 ("Atlanta's", 'NOUN'),
 ('recent', 'ADJ'),
 ('primary', 'NOUN'),
 ('election', 'NOUN'),
 ('produced', 'VERB'),
 ('``', '.'),
 ('no', 'DET'),
 ('evidence', 'NOUN'),
 ("''", '.'),
 ('that', 'ADP'),
 ('any', 'DET'),
 ('irregularities', 'NOUN'),
 ('took', 'VERB'),
 ('place', 'NOUN'),
 ('.', '.')]

In [76]:
# Save the model in the native Keras format
model.save('pos_tagger_model.keras')


In [77]:
import os
os.getcwd()

'C:\\Users\\madas\\Data Science 255 - Batch\\Deep_Learning\\RNN'

In [78]:
import pickle

# Save the word tokenizer
with open('word_tokenizer.pkl', 'wb') as f:
    pickle.dump(tk_x, f)

# Save the tag tokenizer
with open('tag_tokenizer.pkl', 'wb') as f:
    pickle.dump(tk_y, f)